__Обновление:__ Использование Max вместо Mean для объединения эмбедингов

![logo.png](logo.png)

## Предобработка

In [1]:
path = 'data/gazeta_train.jsonl'
vector_size = 300

In [2]:
from datetime import datetime


class Article:
    def __init__(self, date, url, summary, title, text):
        self.date = date
        self.url = url
        self.summary = summary
        self.title = title
        self.text = text


def parse_datetime(string):
  return datetime.strptime(string, '%Y-%m-%d %H:%M:%S')


def parse_article_json(obj):
    date = parse_datetime(obj['date'])
    url = obj['url']
    summary = obj['summary']
    title = obj['title']
    text = obj['text']
    return Article(date, url, summary, title, text)

In [3]:
import json


def parse_gazeta(text):
    lines = text.split('\n')
    articles = [parse_article_json(json.loads(line)) for line in lines if line != '']
    return articles


def load_and_parse_gazeta(path):
    f = open(path, 'r')
    text = f.read()
    return parse_gazeta(text)

In [4]:
articles = load_and_parse_gazeta(path)

In [5]:
import nltk
import re
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer

In [6]:
ru_stopwords = stopwords.words("russian")
morph = MorphAnalyzer()

In [7]:
def preprocess_text(text):
    new_text = text
    new_text = new_text.lower()  # Привести к нижнему регистру
    new_text = re.sub(r'[^\w\s]', ' ', new_text)  # Убрать пунктуацию
    return new_text

In [8]:
def lemmatize_word(word):
    return morph.parse(word)[0].normal_form


def tokenize_sentence(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token for token in tokens if token not in ru_stopwords]  # Убрать стоп слова
    # tokens = [lemmatize_word(token) for token in tokens]
    return tokens


def tokenize_article_body(text):
    sents = nltk.sent_tokenize(text)
    sents = [preprocess_text(sent) for sent in sents]
    sents_words = [tokenize_sentence(sent) for sent in sents]
    return sents_words

In [9]:
len(articles)

60964

In [10]:
sentences = []
for article in articles[:1000]:
    sentences += tokenize_article_body(article.text)

In [11]:
len(sentences)

34113

In [12]:
sentences[0]

['сегодня',
 'транспортный',
 'налог',
 'начисляется',
 'зависимости',
 'мощности',
 'автомобиля',
 'причем',
 'цена',
 'сильных',
 'машин',
 'выше',
 'малолитражек']

## Эмбединг

In [13]:
from gensim.models import FastText, Word2Vec

In [19]:
# ft = FastText(sentences=sentences, vector_size=vector_size, min_count=1, window=5, workers=8)

Лемматизация + Word2Vec очень медленнно, FastText тоже не быстрый. А тут одно из двух, поэтому я выбираю третий вариант: предобученная модель

In [20]:
from navec import Navec

path = 'data/navec_news_v1_1B_250K_300d_100q.tar'
navec = Navec.load(path)

In [21]:
from scipy.spatial import distance
import numpy as np

In [62]:
def vectorize(request):
    tokens = tokenize_text(request)
    vectors = []
    for word in tokens:
        if word in navec:
            vectors.append(navec[word])
    vec_stack = np.stack(vectors)
    vector = np.max(vec_stack, axis=0)
    return vector


distance.cosine(vectorize('взятку'), vectorize('Российский чиновник дал взятку')), \
    distance.cosine(vectorize('взятку'), vectorize('Хакеры взломали пентагон')), \
        distance.cosine(vectorize('взятку'), vectorize('взятки'))

(0.5989381968975067, 0.9491334371268749, 0.342542827129364)

(0.5989381968975067, 0.9491334371268749, 0.342542827129364)

In [43]:
def text_to_vectors(text):
    sents = tokenize_article_body(text)
    vecs = []
    for sent in sents:
        sent_vecs = []
        for token in sent:
            if token in navec:
                sent_vecs.append(navec[token])
        vector = np.zeros((vector_size))
        if len(sent_vecs) > 0:
            vec_stack = np.stack(sent_vecs)
            vector = np.max(vec_stack, axis=0)
        vecs.append(vector)
    return vecs

In [44]:
subsample_size = 1000

In [45]:
for article in articles[:subsample_size]:
    article.summary_vecs = text_to_vectors(article.summary)
    article.title_vecs = text_to_vectors(article.title)
    article.sentences = nltk.sent_tokenize(article.summary)

## Ближайшие соседи и поиск

In [46]:
import annoy

In [47]:
index = annoy.AnnoyIndex(vector_size, 'angular')
counter = 0
for article in articles[:subsample_size]:
    for vec in article.summary_vecs:
        index.add_item(counter, vec)
    for vec in article.title_vecs:
        index.add_item(counter, vec)
    counter+=1

In [48]:
index.build(10, n_jobs=-1)

True

In [58]:
def tokenize_text(text):
    return tokenize_sentence(preprocess_text(text))


def find_nearest_sentence(article, request):
    ind = 0
    min_dist = 1000000
    for i in range(len(article.summary_vecs)):
        dist = distance.cosine(article.summary_vecs[i], request)
        if dist < min_dist:
            min_dist = dist
            ind = i
    return ind


def search(request):
    tokens = tokenize_text(request)
    vector = np.zeros((vector_size))
    vectors = []
    for word in tokens:
        if word in navec:
            vectors.append(navec[word])
    if len(vectors) > 0:
            vec_stack = np.stack(vectors)
            vector = np.max(vec_stack, axis=0)
        # vector /= np.linalg.norm(vector)
    results = index.get_nns_by_vector(vector, 5,)
    return [(i, articles[i].title, \
        articles[i].sentences[find_nearest_sentence(articles[i], vector)]) \
        for i in results]

In [64]:
search('путин')

[(65,
  'Путин прокололся',
  'Mercedes, в котором находился Владимир Путин, попал в ДТП.'),
 (724,
  'Где народ?',
  'Система работает плохо и с перегибами, требуется понять, что именно о гражданах нужно знать органам власти, гласит заявка на научное исследование, заказанное администрацией президента.'),
 (733,
  'Медведеву подарили Путина',
  'Президент России Дмитрий Медведев посетил форум «Селигер» — это первый визит главы государства за всю историю существования «нашистского» проекта.'),
 (48,
  'OneRepublic прохлопали Москву',
  'В Москве выступили американские поп-рокеры OneRepublic — любимцы Тимбаленда вскружили голову своим поклонницам и показали, как работает живьем их машина по зарабатыванию «Грэмми».'),
 (712,
  'О культе Путина',
  'С каждым годом все больше россиян видят в России признаки культа личности Владимира Путина.')]

In [65]:
search('эксперт')

[(388,
  'Россия не отвечает',
  'Россия якобы не ответила ни на один заданный еврочиновниками вопрос, в том числе по делам Магнитского и Ходорковского.'),
 (615,
  'Евро почуял спрос',
  'Дальше будет хуже: инвесторы начинают понимать, что их ожидания по экономическому росту были завышенными.'),
 (276,
  'Европа продолжает снижение',
  'По мнению экспертов, в дальнейшем спад рынка продолжится в связи с завершением стимулирующих программ и нестабильной экономической ситуацией.'),
 (430,
  'Опухли от кризиса',
  'Чудеса по-русски: граждане, согласно официальной статистике, стали жить богаче, чем до кризиса, производство же наверстало кризисные потери едва ли наполовину.'),
 (893,
  '«Денис был просто лучше»',
  'В субботу в немецком Шверине российский боксер Денис Лебедев во втором раунде нокаутировал своего соотечественника Александра Алексеева.')]

In [61]:
search('комментарий')

[(440,
  'Без комментариев',
  'Роскомнадзор вынес первое предупреждение интернет-изданию за комментарии читателей.'),
 (914,
  'Видео придется подождать',
  'Вопрос же о видеоповторах должен подняться на заседании совета в октябре.'),
 (66,
  'На свой медстрах и иск',
  'Он дает гражданам право на выбор страховщика, лечебного учреждения и врача.'),
 (388,
  'Россия не отвечает',
  'Россия якобы не ответила ни на один заданный еврочиновниками вопрос, в том числе по делам Магнитского и Ходорковского.'),
 (824,
  'Послали на три слова',
  'Сенаторы отказались принять документ, так как при передаче текста из одной палаты в другую из него исчезли несколько слов.')]